In [26]:
import redis
import json
from textblob import Word
import re
import nltk
from nltk.corpus import stopwords
import contractions
import string
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
r = redis.Redis(host='redis-12442.c90.us-east-1-3.ec2.cloud.redislabs.com',port=12442,password='RHFgwK2pQ2ibFUqw8zrQBHRzK29lQtDw')
keys=r.keys('*') #getting the keys of each convesation
print(keys)

[b'tracker:e047accea70f4b9ea2a7cf70bbcf8214', b'tracker:1b394dd25cba49c289e1f81458a3f260', b'tracker:6a89a09873af4d5797ada863a91ffd81', b'tracker:e0a88050d69a4968be48de22a81ecaae', b'tracker:ce73a1b0f8f643419969b0086a619f12', b'tracker:f23cbe59dc4947c8a8dc66781fa25c4d', b'tracker:1b0b48d7170a422ba6fe14fb1fdf0385', b'tracker:6079c9d0ef304c67bc67215c87e28bce', b'tracker:8452587c6b5140a08bbaa6806b4856e6', b'tracker:8d72b69da1f34ee8b0707dd0b1a87573', b'tracker:31846644fd194e618f4bc15761e61d05', b'tracker:2259f47dc7c3453e8fdf718fa6ede067', b'tracker:ddf5ac4299414609806dd8018dca0226', b'tracker:0daf13becb784a4980b1d67ea3cf1975', b'tracker:19f68aa275514a60be955ebee8819f8d', b'tracker:0e76f32a85d940d4b498eea26014dda0', b'tracker:446832a509784fbc9ff67dd2547384c3', b'tracker:c03d2a6d67164f38b12a6dfd3f30275e', b'tracker:51862fcd04ac4f0491f701efbdf6c9e0', b'tracker:d1a4fb46f7c842aa9dd0ffa0351807eb', b'tracker:bd09487cb6314e52b2bda5892e8fbb3f', b'tracker:ffc9da9f36c04e1fb618a444d8143982', b'tracker

In [3]:
trackerdata=[]
for i in keys: 
    temp=r[i]
    temp = temp.decode("utf-8")
    trackerdata.append(json.loads(temp)) #each conversation is saved with its events as a list and trackername
conversations=[]
for i in trackerdata:
    conversations.append(i["events"]) #extracting only the events list of each conversation
intents=set()
intentdictnew=dict()
for conv in conversations:
    for i in conv:
        if i['event']=="user":
            if i['parse_data']['intent']['name'] not in intents:
                intents.add(i['parse_data']['intent']['name'])
                intentdictnew[i['parse_data']['intent']['name']]={i["text"]}
            else:
                intentdictnew[i['parse_data']['intent']['name']].add(i["text"])
for k,v in intentdictnew.items():
    print(k,":",v)


price_details : {'price details', 'price detaosl', 'price detaisl', '/price_details'}
get_item_prices : {'/get_item_prices', 'item ifno'}
get_specific_price : {'cost of particular item', '/get_specific_price', 'particular item particular location'}
item_details : {'100185305', '105', '100150000', '1000350001', '100035001', '607'}
location_details : {'loc is 607', 'location 607', 'location 105', 'loc 607', 'price details for location 607'}
get_item_all_locs : {'UPC in RMS', '/get_item_all_locs', 'cost of all items in particular location'}
greet : {"how's the weather", "how's the weather like", 'hey', 'what is atmosphere', 'sup', 'Oracle', 'atmosphere', 'hi'}
user_query : {'/item_details', 'item info]', 'item info'}
location_info : {'location 246001', '100035001', '100150000'}
hello : {'/hello'}
nlu_fallback : {'prime minister of india', 'what is climatic change', 'who is vendor', 'UPC in retail', 'president of india', 'rasa', 'list IT companies chennai', 'what is the wetaher like', '/ac

DESCRIPTIVE ANALYSIS OF UNRECOGNISED USER INPUTS

In [40]:
print(intentdictnew["nlu_fallback"])
print(intentdictnew["others"])
newinputs=intentdictnew["nlu_fallback"] | intentdictnew["others"]
print(newinputs)

{'prime minister of india', 'what is climatic change', 'who is vendor', 'UPC in retail', 'president of india', 'rasa', 'list IT companies chennai', 'what is the wetaher like', '/action_get_item_prices', 'what is UPC', '100035001', 'UPC', 'what is the weather condition in trichy', '607', 'loc is 607', '100150000', 'what is RASA', 'prime minister of tamilnadu', 'who is the cheoif minister of tamilnadu', 'cheif minister of tamilnadu'}
{'what is the weather', 'hwta is UPC', 'what is python', '/others', 'who is the cheif minister of tamilnadu', 'what is the weather in trichy', 'what is the weather like', 'what is the weather in chennai', 'what is RMS'}
{'UPC', 'what is the weather condition in trichy', 'prime minister of india', 'what is climatic change', 'what is the weather', 'what is python', '607', 'what is the weather in trichy', 'who is vendor', 'UPC in retail', 'president of india', 'rasa', 'list IT companies chennai', 'what is the wetaher like', '/action_get_item_prices', 'loc is 60

PREPROCESSING THE DATA

In [42]:
#removing button inputs
newinputs2=set()
for i in newinputs:
    if i[0]!='/':
        newinputs2.add(i.lower())
newinputs2=list(newinputs2)
print(newinputs2)

['prime minister of india', 'what is climatic change', 'what is python', 'what is the weather in trichy', 'who is vendor', 'list it companies chennai', 'president of india', 'rasa', 'what is rasa', 'what is the wetaher like', 'what is rms', '100035001', 'what is upc', 'who is the cheif minister of tamilnadu', 'what is the weather like', 'what is the weather in chennai', 'what is the weather condition in trichy', 'what is the weather', '607', 'upc', 'upc in retail', 'loc is 607', '100150000', 'hwta is upc', 'prime minister of tamilnadu', 'who is the cheoif minister of tamilnadu', 'cheif minister of tamilnadu']


In [30]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)

In [31]:
def cleaning_contractions(text):
    text=text.split()
    corrected=[]
    for word in text:
        corrected.append(contractions.fix(word))
    return " ".join(corrected)

In [32]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    return re.sub(re.compile("["+punctuations_list+"‼️’]")," ",text)

In [33]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def cleaning_stopwords(data):
    words = data.split()
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\metul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def correct_word_spelling(sentence):
    words=sentence.split()
    words2=[]
    for word in words:
        word = Word(word)
        words2.append(word.correct())
    return(" ".join(words2))

In [43]:
userinputs=[]
for i in newinputs2:
    i=cleaning_numbers(i)
    i=cleaning_contractions(i)
    i=cleaning_punctuations(i)
    i=cleaning_stopwords(i)
    #i=correct_word_spelling(i)
    if i!="":
        print(i)
        userinputs.append(i)

prime minister india
climatic change
python
weather trichy
vendor
list companies chennai
president india
rasa
rasa
wetaher like
rms
upc
cheif minister tamilnadu
weather like
weather chennai
weather condition trichy
weather
upc
upc retail
loc
hwta upc
prime minister tamilnadu
cheoif minister tamilnadu
cheif minister tamilnadu


In [44]:
print(userinputs)

['prime minister india', 'climatic change', 'python', 'weather trichy', 'vendor', 'list companies chennai', 'president india', 'rasa', 'rasa', 'wetaher like', 'rms', 'upc', 'cheif minister tamilnadu', 'weather like', 'weather chennai', 'weather condition trichy', 'weather', 'upc', 'upc retail', 'loc', 'hwta upc', 'prime minister tamilnadu', 'cheoif minister tamilnadu', 'cheif minister tamilnadu']


In [45]:
words=[]
for i in userinputs:
    words.extend(i.split())
print(words)
words=Counter(words)
print(words)

['prime', 'minister', 'india', 'climatic', 'change', 'python', 'weather', 'trichy', 'vendor', 'list', 'companies', 'chennai', 'president', 'india', 'rasa', 'rasa', 'wetaher', 'like', 'rms', 'upc', 'cheif', 'minister', 'tamilnadu', 'weather', 'like', 'weather', 'chennai', 'weather', 'condition', 'trichy', 'weather', 'upc', 'upc', 'retail', 'loc', 'hwta', 'upc', 'prime', 'minister', 'tamilnadu', 'cheoif', 'minister', 'tamilnadu', 'cheif', 'minister', 'tamilnadu']
Counter({'minister': 5, 'weather': 5, 'upc': 4, 'tamilnadu': 4, 'prime': 2, 'india': 2, 'trichy': 2, 'chennai': 2, 'rasa': 2, 'like': 2, 'cheif': 2, 'climatic': 1, 'change': 1, 'python': 1, 'vendor': 1, 'list': 1, 'companies': 1, 'president': 1, 'wetaher': 1, 'rms': 1, 'condition': 1, 'retail': 1, 'loc': 1, 'hwta': 1, 'cheoif': 1})
